This Lab is made with inspiration & code snippets of:
- https://github.com/yxtay/char-rnn-text-generation
- https://d2l.ai/chapter_recurrent-modern/lstm.html
- https://d2l.ai/chapter_recurrent-neural-networks/rnn-scratch.html

In [1]:
import mxnet as mx
import mxnet.ndarray as F
import mxnet.gluon as gluon
from mxnet.gluon import nn, rnn
from mxnet import autograd

import time
from tqdm import tqdm

from utils.text_helpers import load_corpus, Vocab, batch_generator, encode_text, decode_text, generate_seed, sample_from_probs

In [2]:
class Model(gluon.Block):
    """
    build character embeddings LSTM text generation model.
    """
    def __init__(self, vocab_size, embedding_size=32,
                 rnn_size=128, num_layers=2, drop_rate=0.0, **kwargs):
        super(Model, self).__init__(**kwargs)

        self.args = {
            "vocab_size": vocab_size,
            "embedding_size": embedding_size,
            "rnn_size": rnn_size,
            "num_layers": num_layers,
            "drop_rate": drop_rate}

        with self.name_scope():
            self.encoder = nn.Embedding(vocab_size, embedding_size)
            self.dropout = nn.Dropout(drop_rate)
            self.rnn = rnn.LSTM(rnn_size, num_layers, dropout = drop_rate,
                                input_size = embedding_size)
            self.decoder = nn.Dense(vocab_size, in_units = rnn_size)

    def forward(self, inputs, state):
        # input shape: [seq_len, batch_size]
        seq_len, batch_size = inputs.shape
        embed_seq = self.dropout(self.encoder(inputs))
        # shape: [seq_len, batch_size, embedding_size]
        rnn_out, state = self.rnn(embed_seq, state)
        # rnn_out shape: [seq_len, batch_size, rnn_size]
        # hidden shape: [2, num_layers, batch_size, rnn_size]
        rnn_out = self.dropout(rnn_out)
        # shape: [seq_len, batch_size, rnn_size]
        logits = (self.decoder(rnn_out.reshape((-1, rnn_out.shape[2])))
                  .reshape((seq_len, batch_size, -1)))
        # output shape: [seq_len, batch_size, vocab_size]
        return logits, state

    def begin_state(self, batch_size=1):
        """
        initialises rnn states.
        """
        return self.rnn.begin_state(batch_size)

In [3]:
def train(corpus, vocab, embedding_size=32,
          rnn_size=128, num_layers=2, drop_rate=0.0,
          batch_size = 64, seq_len = 64, num_epochs = 64):
    """
    trains model specfied in args.
    main method for train subcommand.
    """

    print("corpus length: %s, vocabulary size: %s" %(len(corpus), len(vocab)))

    VOCAB_SIZE = len(vocab)

    model = Model(vocab_size=VOCAB_SIZE,
                  embedding_size=embedding_size,
                  rnn_size=rnn_size,
                  num_layers=num_layers,
                  drop_rate=drop_rate)
    model.initialize(mx.init.Xavier())
    model.hybridize()

    # loss function
    loss = gluon.loss.SoftmaxCrossEntropyLoss(batch_axis=1)

    # optimizer
    optimizer = mx.optimizer.Adam(learning_rate=0.001, clip_gradient=5.0)

    # trainer
    trainer = gluon.Trainer(model.collect_params(), optimizer)

    # training start
    num_batches = (len(corpus) - 1) // (batch_size * seq_len)
    data_iter = batch_generator(encode_text(corpus, char2id=vocab), batch_size = batch_size, seq_len=seq_len, vocab = vocab)
    state = model.begin_state(batch_size)

    print("start of training.")
    time_train = time.time()
    for i in range(num_epochs):
        epoch_losses = mx.nd.empty(num_batches)
        time_epoch = time.time()
        # training epoch
        for j in tqdm(range(num_batches), desc="epoch {}/{}".format(i + 1, num_epochs), position=0, leave=True):
            # prepare inputs
            x, y = next(data_iter)
            x = mx.nd.array(x.T)
            y = mx.nd.array(y.T)
            # reset state variables to remove their history
            state = [arr.detach() for arr in state]

            with autograd.record():
                logits, state = model(x, state)
                # calculate loss
                L = loss(logits, y)
                L = F.mean(L)
                epoch_losses[j] = L.asscalar()
                # calculate gradient
                L.backward()
            # apply gradient update
            trainer.step(1)

        # logs
        duration_epoch = time.time() - time_epoch
        print("epoch: %s, duration: %ds, loss: %.6g."
              %(i + 1, duration_epoch, F.mean(epoch_losses).asscalar()))

        # generate text
        seed = generate_seed(corpus)
        generate_text(model, seed, vocab=vocab)

    # training end
    duration_train = time.time() - time_train
    print("end of training, duration: %ds." %duration_train)
    # generate text
    seed = generate_seed(corpus)
    generate_text(model, seed, 1024, 3, vocab=vocab)
    return model

In [4]:
def generate_text(model, seed, length=256, top_n=4, vocab=Vocab):
    """
    generates text of specified length from trained model
    with given seed character sequence.
    """
    print("generating %s characters from top %s choices."%(length, top_n))
    print('generating with seed: "%s".' % (''.join(seed))) # ['a', 'b', 'c'] -> "abc"

    generated = seed
    encoded = mx.nd.array(encode_text(seed, char2id=vocab))
    seq_len = encoded.shape[0]

    x = F.expand_dims(encoded[:seq_len-1], 1)
    # input shape: [seq_len, 1]
    state = model.begin_state()
    # get rnn state due to seed sequence
    _, state = model(x, state)

    next_index = encoded[seq_len-1].asscalar()
    for i in range(length):
        x = mx.nd.array([[next_index]])
        # input shape: [1, 1]
        logit, state = model(x, state)
        # output shape: [1, vocab_size]
        probs = F.softmax(logit)
        next_index = sample_from_probs(probs.asnumpy().squeeze(), top_n)
        # append to sequence
        generated += vocab.to_tokens(next_index) #ID2CHAR[next_index]

    print("generated text: \n%s\n" %(''.join(generated)))
    return generated

In [5]:
# read text
corpus, vocab = load_corpus('data/tinyshakespeare.txt', token_type = 'char')

# corpus, vocab = load_corpus('data/tinyshakespeare.txt')

In [7]:
vocab._token_to_idx

{'<unk>': 0,
 ' ': 1,
 'e': 2,
 't': 3,
 'o': 4,
 'a': 5,
 'h': 6,
 's': 7,
 'r': 8,
 'n': 9,
 'i': 10,
 '\n': 11,
 'l': 12,
 'd': 13,
 'u': 14,
 'm': 15,
 'y': 16,
 ',': 17,
 'w': 18,
 'f': 19,
 'c': 20,
 'g': 21,
 'I': 22,
 'b': 23,
 'p': 24,
 ':': 25,
 '.': 26,
 'A': 27,
 'v': 28,
 'k': 29,
 'T': 30,
 "'": 31,
 'E': 32,
 'O': 33,
 'N': 34,
 'R': 35,
 'S': 36,
 'L': 37,
 'C': 38,
 ';': 39,
 'W': 40,
 'U': 41,
 'H': 42,
 'M': 43,
 'B': 44,
 '?': 45,
 'G': 46,
 '!': 47,
 'D': 48,
 '-': 49,
 'F': 50,
 'Y': 51,
 'P': 52,
 'K': 53,
 'V': 54,
 'j': 55,
 'q': 56,
 'x': 57,
 'z': 58,
 'J': 59,
 'Q': 60,
 'Z': 61,
 'X': 62,
 '3': 63,
 '&': 64,
 '$': 65}

In [8]:
model = train(corpus, vocab, drop_rate=0.2)

corpus length: 1115394, vocabulary size: 66
start of training.


epoch 1/64:   0%|          | 1/272 [00:00<00:37,  7.14it/s]

number of batches: 272.
effective text length: 1114112.
x shape:  (64, 17408)
y shape:  (64, 17408)


epoch 1/64: 100%|██████████| 272/272 [00:53<00:00,  5.05it/s]


epoch: 1, duration: 53s, loss: 2.99956.
generating 256 characters from top 4 choices.
generating with seed: "u ".
generated text: 
u thee he tiud theure theur het art that ther heus sath an want at seos andess wo hin son sas hand, wo has tor af sit the so seat thore hise the hith and sise he mot sore sand, the we sase thise, than toor ther the wore te that soud with tin hith, ard sore t



...

epoch: 62, duration: 66s, loss: 1.36639.
generating 256 characters from top 4 choices.
generating with seed: "rs".
generated text: 
rsO Off my friend,
And, and heavy to still to me to beast,
I am all all this was any more of their,
Were some bread as is the seasons of anger.

GEORGE:
I had not will bear my contents. Would he should be
not but the power, and the way with an hand
As thou a



epoch 63/64: 100%|██████████| 272/272 [01:08<00:00,  4.00it/s]


epoch: 63, duration: 68s, loss: 1.3651.
generating 256 characters from top 4 choices.
generating with seed: " is much".
generated text: 
 is much one and so more art at the way,
We will, we were a service of a madal:
I'll be honour, to the winds, what have stoned
The sententation,--

ANTONIO:
Ther to him and here as you will be honour,
Where the preserve of her heart that have
A fearth of me to him



epoch 64/64: 100%|██████████| 272/272 [01:03<00:00,  4.28it/s]


epoch: 64, duration: 63s, loss: 1.36323.
generating 256 characters from top 4 choices.
generating with seed: "ot?
".
generated text: 
ot?

BIANCA:
I will be speak in my last of her hands,
That hours that we hold her the maid of hands,
To the supposed alive of my hand to myself
I shall be too subjects and thine eyes,
And then altorious man these hast things to him.
Why, the duke of York;
Thin

end of training, duration: 4502s.
generating 1024 characters from top 3 choices.
generating with seed: "saw him.

QUEEN ".
generated text: 
saw him.

QUEEN MARGARET:
Ay, thou hast spoke the sun of the such dear sight:
The stoop thou holled to thee, talk'd and so despase this strength,
The wind to the proud sorrow will no more as the men
to the court of thee, and the world all the state.

LUCIO:
I'll still, where in a merchant were to say. I am, true. 
ANTONIO:
A more of the will the statious true.

BIONDELLO:
The watch the contrary were so that she seems
I warrant the war and his brains of him.

K

generating with seed: "enoughthe great ".
generated text:
enoughthe great oe ote te t e oe teetit a te th ae o at teen te aee th t aae ioe aa t aet tee te th thh thie tethete  aoeeee aa teet et ee o th tei aoe aenteen  th t ae ath ee iaie teetee tete thee tohi oee a iote e oaee aa te aathee to e ooe to o ote  ohh aa o oeenet ate

In [16]:
# model, seed, length=512, top_n=10, vocab=Vocab
generate_text(model, "KING RICHARD: ", length=512, top_n=4, vocab=vocab)

generating 512 characters from top 4 choices.
generating with seed: "KING RICHARD: ".
generated text: 
KING RICHARD: what
think, to see the pleasures talk of the maid,
And that his sorrow opposed and my life,-
Within my soul is true of this wrive a way,
The will be a mean woman in the world
Though he shall not seek to take all at this.
Why tell me, to me and think on how are,
By her hand of thy father of his head.
I hope, as they are true, sir. Think him that mean
That word to the steal is a thousand man.

LADY GREY:
Why, sir, my soul to that, and see hide told these
As was the south of his crown on that treason,
Which wo



'KING RICHARD: what\nthink, to see the pleasures talk of the maid,\nAnd that his sorrow opposed and my life,-\nWithin my soul is true of this wrive a way,\nThe will be a mean woman in the world\nThough he shall not seek to take all at this.\nWhy tell me, to me and think on how are,\nBy her hand of thy father of his head.\nI hope, as they are true, sir. Think him that mean\nThat word to the steal is a thousand man.\n\nLADY GREY:\nWhy, sir, my soul to that, and see hide told these\nAs was the south of his crown on that treason,\nWhich wo'